### SOME NOTES

- To improve performance, check https://svail.github.io/rnn_perf/

    - For readers in first group (users of deep learning frameworks) - the main takeaway is to make layer sizes and mini-batch sizes multiples of 32, and if you're using cuBLAS, then make them multiples of 64 for best performance. If you're writing recurrent layers yourself, make sure to write them in such a way that you combine across time when possible. 
    
- Try to hyperparameter optimization: http://maxpumperla.com/hyperas/

- SincNet can be helpful. https://github.com/mravanelli/SincNet/

- Check the transformer. https://jalammar.github.io/illustrated-transformer/ https://ai.googleblog.com/2017/08/transformer-novel-neural-network.html http://nlp.seas.harvard.edu/2018/04/03/attention.html

- Autokeras. https://github.com/jhfjhfj1/autokeras

- Should we use FeedForward? https://www.offconvex.org/2018/07/27/approximating-recurrent/

- TCN https://github.com/philipperemy/keras-tcn

- Another method for speaker change detection https://github.com/philipperemy/speaker-change-detection

- Rare sound event detection: https://www.cs.tut.fi/sgn/arg/dcase2017/documents/challenge_technical_reports/DCASE2017_Lim_204.pdf

- We can augment data via [Pydub](https://github.com/jiaaro/pydub). https://towardsdatascience.com/tensorflow-speech-recognition-challenge-solution-outline-9c42dbd219c9

- Convolutional Recurrent Neural Networks for Small Footprint Keyword Spotting https://arxiv.org/pdf/1703.05390.pdf

- CNN and Speaker Change Detection http://mirlab.org/conference_papers/International_Conference/ICASSP%202017/pdfs/0004945.pdf

- Different models for speech recognition https://github.com/lucko515/speech-recognition-neural-network/blob/master/vui_notebook.ipynb
-----------------------

### TO-DO

- Generator modelini implement et.

- Visualize the system. https://musicinformationretrieval.com 
    - We can use https://github.com/stevetjoa/musicinformationretrieval.com/blob/gh-pages/onset_segmentation.ipynb
    

- Augment the input data.

- Try CNN, CapsuleNet and Neural Turing Machine.

- Fuzzy Labeling 
    - [We](http://mirlab.org/conference_papers/International_Conference/ICASSP%202017/pdfs/0004945.pdf) developed a new labeling strategy to cope with the uncertainty introduced by the human annotators.  Instead of the
previously used binary labeling we use a fuzzy labeling. The
value one is assigned when the speaker change is located in
the middle of the analyzed window and linearly decreases to
zero as the change moves away from the middle.

-------------

In this notebook, we are trying to reproduce [the paper](https://pdfs.semanticscholar.org/edff/b62b32ffcc2b5cc846e26375cb300fac9ecc.pdf) for speaker change detection

**TODO**

- Use AMI Corpus with feature extraction with pyannote.

## Review

**Sequence Labelling** 

They think this task as a binary classification. Thus, they label changing frame as a **1** and non-changing frame as a **0**. So that, they use the _binary cross-entropy loss function_.

**Network Architecture**
- 2 Bi-LSTM
    - 64 and 32 outputs respectively.
- Multi Layer Perceptron
    - 3 Fully Connected Feedforward Layers
        - 40, 20, 1 dimensional respectively.
    - Tanh activation for first 2 layer
    - Sigmoid activation for last layer
    
**Feature Extraction**
- "35-dimensional acoustic features are extracted every 16ms on a 32ms window using [Yaafe toolkit](http://yaafe.sourceforge.net)."
    - 11 Mel-Frequency Cepstral Coefficients (MFCC), 
    - Their first and second derivatives,
    - First and second derivatives of the energy.

**Class Imbalance**

- _"The number of positive labels isincreased artificially by labeling as positive every frame in the direct neighborhood of the manually annotated change point."_
- A positive neighborhood of 100ms (50ms on both sides) is used around each change point, to partially solve the class imbalance problem.

**Subsequences**

- _"The long audio sequences are split into short fixed-length overlapping sequences."_

**Prediction**

- _"Finally, local score maxima exceeding a pre-determined threshold θ are marked as speaker change points."_

**Training**

- Subsequences for training are 3.2s long with a step of 800ms (i.e. two adjacent sequences overlap by 75%).

**From Codebase**

- For Optimizer: _SGD, momentum=0.9, nesterov=True_

- Architecture:
       name: StackedRNN
       params:
         rnn: LSTM
         recurrent: [32, 20]
         bidirectional: True
         linear: [40, 10]
         
-  create final classification layer (with log-softmax activation)
    - NLLLoss

## Code

### Feature Extraction

We will use Yaafe Toolkit. (To see all available features, you can use _!yaafe -l_) To learn how we can do that, start with http://yaafe.github.io/Yaafe/manual/quickstart.html#quick-start-using-yaafe


In [ ]:
# You can view a description of each feature (or output format) with the -d option:

# !yaafe -d MFCC

In [ ]:
# !yaafe -d Energy

Let's determine blockSize and stepSize. 

If we have 16kHz audio signal(in AMI, we have 16kHz), for 32 ms block, we need 16x32, For the stepsize as 16 ms, we need 16x16 size.

We need these features:

- mfcc: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11
- mfcc_d1: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=1
- mfcc_d2: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=2
- energy_d1: Energy blockSize=512 stepSize=256  > Derivate DOrder=1
- energy_d2: Energy blockSize=512 stepSize=256  > Derivate DOrder=2

To extract all of these, we will use [this technique](http://yaafe.github.io/Yaafe/manual/quickstart.html#extract-several-features). Shortly, we will write all these features into single text file.

In [ ]:
# If you want to use 32 ms window length with 16 ms hop
# This feature extraction is based on paper

f = open("featureplan.txt", "w")
f.write("mfcc: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 \n"
        "mfcc_d1: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=1 \n"
        "mfcc_d2: MFCC blockSize=512 stepSize=256 CepsNbCoeffs=11 > Derivate DOrder=2 \n"
        "energy_d1: Energy blockSize=512 stepSize=256  > Derivate DOrder=1 \n"
        "energy_d2: Energy blockSize=512 stepSize=256  > Derivate DOrder=2")
f.close() 

In [ ]:
# If you want to use 25 ms window length with 10 ms hop
# This feature extraction is based on pyannote library

f = open("featureplan_new.txt", "w")
f.write("mfcc: MFCC blockSize=400 stepSize=160 CepsNbCoeffs=19 \n"
        "mfcc_d1: MFCC blockSize=400 stepSize=160 CepsNbCoeffs=19 > Derivate DOrder=1 \n"
        "mfcc_d2: MFCC blockSize=400 stepSize=160 CepsNbCoeffs=19 > Derivate DOrder=2 \n"
        "energy_d1: Energy blockSize=400 stepSize=160 > Derivate DOrder=1 \n"
        "energy_d2: Energy blockSize=400 stepSize=160 > Derivate DOrder=2")
f.close() 

In [ ]:
# Now, you can test the yaafe with following command

# !yaafe -c "featureplan_new.txt" -r 16000 "ES2009a.Mix-Headset.wav" -p Precision=6 -p Metadata=False -n

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

from numpy import genfromtxt
import numpy as np
import os
import glob

import librosa
import os
import glob
import numpy as np
import math
import sys

def create_numpy_for_audio(audio_file, featureplan = "", feature_extractor="yaafe", hop=10, win_len=25, sr=16000):
    """This function is based on YAAFE or Librosa. It will return 2D Array which is features of audio file. 
    Also it will save the numpy array.
    
    Its arguments:
    audio_file: Path of audio file, it can be wav, mp3, ogg etc.
    featureplan: Text file which introduce which features will be extracted.(for Yaafe)
    feature_extractor: Which library will be used to extract features.
    hop: Hop length (we need it for Librosa)
    win_len: Window length (we need it for Librosa)
    sr: Sample rate for audio file. Default as 16000."""
    
    if (feature_extractor=="yaafe"):
        !yaafe -c $featureplan -r $sr $audio_file -p Precision=6 -p Metadata=False -n
        filename = (audio_file.split("/")[-1]).split(".")[0]

        my_data = genfromtxt(audio_file + ".mfcc.csv", delimiter=',')
        my_data = np.append(my_data, genfromtxt(audio_file + ".mfcc_d1.csv", delimiter=','), axis=1)
        my_data = np.append(my_data, genfromtxt(audio_file + ".mfcc_d2.csv", delimiter=','), axis=1)

        my_data = np.append(my_data, np.expand_dims(genfromtxt(audio_file + ".energy_d1.csv", delimiter=','), axis=1), axis=1)
        my_data = np.append(my_data, np.expand_dims(genfromtxt(audio_file + ".energy_d2.csv", delimiter=','), axis=1), axis=1)

        # Previous codes creates csv file for features to load numpy array. After that, we can 
        # remove them.
        os.remove(audio_file + ".mfcc.csv")
        os.remove(audio_file + ".mfcc_d1.csv")
        os.remove(audio_file + ".mfcc_d2.csv")
        os.remove(audio_file + ".energy_d1.csv")
        os.remove(audio_file + ".energy_d2.csv")
        
        np.save("./yaafe_ami_storage/" + filename, my_data)

        return my_data
    
    if (feature_extractor=="librosa"):
        audio, sr = librosa.load(filename)
        
        # https://github.com/librosa/librosa/issues/584
        mfccs = librosa.feature.mfcc(audio, sr, n_mfcc=11, hop_length=int(float(hop/1000)*sr), n_fft=int(float(win_len/1000)*sr))
        mfccs_d1 = librosa.feature.delta(mfccs)
        mfccs_d2 = librosa.feature.delta(mfccs, order=2)
        energy = librosa.feature.rmse(y=audio, hop_length=int(float(hop/1000)*sr), frame_length=int(float(win_len/1000)*sr))
        energy_d1 = librosa.feature.delta(energy)
        energy_d2 = librosa.feature.delta(energy, order=2)

        my_data = np.vstack((mfccs, mfccs_d1, mfccs_d2, energy_d1, energy_d2))
        
        np.save("./librosa_ami_storage/" + filename, my_data)
        
        return my_data


To understand, how normalization and delta features affect performance https://ieeexplore.ieee.org/document/5711789/

In [ ]:
import math
import numpy as np
import glob
import os
import matplotlib.pyplot as pp
import more_itertools as mit

%matplotlib inline

def create_data_for_supervised(root_dir, hop, win_len, from_ep=0, to_ep=0, boost_for_imbalance=False, 
                              how_much_boost=6, balance=False, 
                              feature_extractor="pyannote",
                              fuzzy_label=False):
    
    """If we have numpy array in the folder, it will create input and output array via this function.
    
    Arguments:
    root_dir: The folder which stores numpy array.
    hop: Hop length (we need it for Librosa).
    win_len: Window length (we need it for Librosa).
    from_ep: Location of first file which will be loaded into array.
    to_ep: Location of last file which will be loaded into array.
    boost_for_imbalance: If it is true, the number of positive labels is
        increased artificially by labeling as positive every frame in the
        direct  neighborhood  of  the  manually  annotated  change  point
        and this number will be determined by how_much_boost parameter.
    balance: if it is true, we will discard some frame which are at the 
        middle of single speaker segment.
    feature_extractor: Which library will be used. (now, we can use Pyannote or Yaafe.)
    fuzzy_label: If it is true, we will label the neighboorhood frames
        with decreasing number. (for instance: if we will use 5 frame as a boost
        middle frame will be 1, next frame will be 0.8, next one will be 0.6)
    """
    
    audios_paths = glob.glob(os.path.join(root_dir, '*wav'))
    input_array = []
    
    output_array = []
    
    num = 0
    
    for single_audio_path in audios_path:
        
        num += 1
        
        if ((num >= from_ep) and (num < to_ep)):
            
            change_seconds = []

            filename = (single_audio_path.split("/")[-1]).split(".")[0]
            print (filename)
            
            try:
                
                if (feature_extractor == "pyannote"):
                    feature_vector = np.load("./pyannote-audio/tutorials/feature-extraction/AMI/" + filename + ".Mix-Headset.npy")
                    
                if (feature_extractor == "yaafe"):
                    feature_vector = np.load("./yaafe_ami_storage/" + filename + ".Mix-Headset.npy")
                    
                    print (feature_vector.shape)
                    
                feature_array = np.ravel(feature_vector)

                if (feature_vector is not None):

                    input_array.extend(feature_array)
                    
                    print (single_audio_path + " is done.")

                    path_for_output = "./txt_ami_full/" + filename + "_full_time.txt"

                    with open(path_for_output) as f:
                        content = f.readlines()


                    # need to open text file
                    # after that, point the end point of speaker
                    # add 1 to point of speaker, add 0 otherwise
                    # time is in second format at the txt file
                    
                    content = [x.strip() for x in content] 

                    for single_change in content:

                        change_seconds.append(single_change)

                    output_array = np.zeros(feature_vector.shape[0])

                    for single_change in change_seconds:
                        
                        single_change_ms = float(single_change)*1000
                        
                        which_start_hop = (single_change_ms-win_len)/hop # now we know, milisecond version of change
                                                    # which is located after which_hop paramater
                                                    # add 2 and round to up
                                
                        which_end_hop = single_change_ms/hop # round to up

                        start_location = math.ceil(which_start_hop + 1)
                        end_location = math.ceil(which_end_hop)

                        if (boost_for_imbalance==False):
                            
                            feature_array[start_location:end_location+1] = 1.0

                        else:
                            
                            if (fuzzy_label==False):
                                feature_array[start_location-how_much_boost:end_location+1+how_much_boost] = 1.0
                            else:
                                feature_array[start_location:end_location+1] = 1.0
                                for ix_label in range(1, how_much_boost):
                                    feature_array[start_location-ix_label] = 1.0 - (float(ix_label)/how_much_boost)
                                    feature_array[end_location+1+ix_label] = 1.0 - (float(ix_label)/how_much_boost)
                                    
                    output_array.extend(feature_array)
                    
            except:
                print ("Pass this file...")
                pass
 
            
    input_array = np.reshape(input_array, (-1, 59))
        
    output_array = np.asarray(output_array)
    output_array = np.expand_dims(output_array, axis=1)
    
    if (balance == True):
            loc_zeros = np.where(output_array == 0)[0]
            list_cons = [list(group) for group in mit.consecutive_groups(loc_zeros)]
            
            for single_list_con in list_cons:
                if (len(single_list_con) > 80):
                    first_zero_loc = single_list_con[0]
                    last_zero_loc = single_list_con[-1]
                    output_all_array[first_zero_loc+20:last_zero_loc-20] = 2
                    
            loc_twos = np.where(output_array == 2)[0]
            list_cons = [list(group) for group in mit.consecutive_groups(loc_twos)]
            
            output_array = output_array.squeeze(axis=1)
            input_array = input_array[output_array != 2, :]
            
            output_array = output_array[output_all_array != 2]   
            output_array = np.expand_dims(output_array, axis=1)
            
    
    print("inputs shape: ", input_array.shape)

    print("outputs shape: ", output_array.shape)

    return (input_array, output_array)

--------

## Build Architectures and Train Them

**2D Convolution and BiLSTM Combination**

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import keras

from keras import layers
from keras import models
from keras import optimizers
from keras.models import Model
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.models import load_model
import time

frame_shape = (800, 59, 1)

## Network Architecture
# https://keras.io/getting-started/sequential-model-guide/

input_frame = keras.Input(frame_shape, name='main_input')

conv1 = layers.Conv2D(800, 9, strides=(1,1), padding="same")(input_frame)
conv1 = layers.LeakyReLU()(conv1)
conv1_BN = layers.BatchNormalization()(conv1)
conv1_drop = layers.Dropout(0.5)(conv1_BN)

xx = layers.TimeDistributed(layers.Flatten())(conv1_drop)

bidirectional_1 = layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_initializer='random_normal',
                bias_initializer='zeros'))(xx)
bidirectional_1 = layers.LeakyReLU()(bidirectional_1)
bidirectional_1_BN = layers.BatchNormalization()(bidirectional_1)
bidirectional_1_drop = layers.Dropout(0.5)(bidirectional_1_BN)

bidirectional_2 = layers.Bidirectional(layers.LSTM(64, return_sequences=True,  kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_1_drop)
bidirectional_2 = layers.LeakyReLU()(bidirectional_2)
bidirectional_2_BN = layers.BatchNormalization()(bidirectional_2)
bidirectional_2_drop = layers.Dropout(0.5)(bidirectional_2_BN)

bidirectional_3 = layers.Bidirectional(layers.LSTM(36, return_sequences=True,  kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_2_drop)
bidirectional_3 = layers.LeakyReLU()(bidirectional_3)
bidirectional_3_BN = layers.BatchNormalization()(bidirectional_3)
bidirectional_3_drop = layers.Dropout(0.5)(bidirectional_3_BN)


tdistributed_1 = layers.TimeDistributed(layers.Dense(40, kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_3_drop)
tdistributed_1 = layers.LeakyReLU()(tdistributed_1)
tdistributed_1_BN = layers.BatchNormalization()(tdistributed_1)
tdistributed_1_drop = layers.Dropout(0.5)(tdistributed_1_BN)

tdistributed_2 = layers.TimeDistributed(layers.Dense(10, kernel_initializer='random_normal',
                bias_initializer='zeros'))(tdistributed_1_drop)
tdistributed_2 = layers.LeakyReLU()(tdistributed_2)
tdistributed_2_BN = layers.BatchNormalization()(tdistributed_2)
tdistributed_2_drop = layers.Dropout(0.5)(tdistributed_2_BN)


tdistributed_3 = layers.TimeDistributed(layers.Dense(1, activation='sigmoid', kernel_initializer='random_normal',
                bias_initializer='zeros'))(tdistributed_2_drop)


# WE DO NOT NEED IT FOR TRAINING. SO DISCARD.
## Source: https://stackoverflow.com/questions/37743574/hard-limiting-threshold-activation-function-in-tensorflow
def step_activation(x):
    threshold = 0.4
    cond = tf.less(x, tf.fill(value=threshold, dims=tf.shape(x)))
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x)))

    return out

# https://stackoverflow.com/questions/47034692/keras-set-output-of-intermediate-layer-to-0-or-1-based-on-threshold

step_activation = layers.Dense(1, activation=step_activation, name='threshold_activation')(tdistributed_3)



model = Model(input_frame, tdistributed_3)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=0.0001, decay=0.00001)

Nadam = keras.optimizers.Nadam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=0.00001, schedule_decay=0.0004)

model.compile(loss='binary_crossentropy', optimizer="Nadam")

In [ ]:
model.summary()

In [ ]:
how_many_step = 40
how_many_repeat = 30

ix_repeat = 0


while (ix_repeat < how_many_repeat):
    ix_repeat += 0
    
    print ("REPEAT:", ix_repeat)
    ix_step = 0
    from_epi = 1
    
    while (ix_step < how_many_step):
        ix_step += 1

        print ("STEP:", ix_step)
        
        #print("relax")
        #time.sleep(2.5) 

        input_array, output_array = create_data_for_supervised ("./amicorpus/*/audio/", 10, 25, from_epi, 
                                                                from_epi+3, True, 12, False, "yaafe" , False, True)
        
        print (np.mean(output_array))
        max_len = 800 # how many frame will be taken
        step = 800 # step size.

        input_array_specified = []
        output_array_specified = []

        for i in range (0, input_array.shape[0]-max_len, step):
            single_input_specified = (input_array[i:i+max_len,:])
            single_output_specified = (output_array[i:i+max_len,:])

            input_array_specified.append(single_input_specified)
            output_array_specified.append(single_output_specified)

        output_array_specified = np.asarray(output_array_specified)
        input_array_specified = np.asarray(input_array_specified)

        try:
            input_array_specified = np.expand_dims(input_array_specified, axis=4)
            model.fit(input_array_specified, output_array_specified,
                   epochs=2,
                   batch_size=16,
                   shuffle=True)

        except:
            print ("Pass this epoch.")
            pass


        model.save_weights('bilstm_weights_2DCNN.h5')    

        input_array = []
        output_array = []

        from_epi += 3
    model.save_weights("bilstm_weights_2DCNN" + str(ix_repeat) + ".h5")    

**BiLSTM combined with Attention**

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

from keras import layers
from keras import models
from keras import optimizers
import keras
from keras.models import load_model
import time
from keras.models import Model
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects


frame_shape = (800, 59)

## Network Architecture

input_frame = keras.Input(frame_shape, name='main_input')

bidirectional_1 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(input_frame)
bidirectional_2 = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=True))(bidirectional_1)

# compute importance for each step
attention = layers.Dense(1, activation='tanh')(bidirectional_2)
attention = layers.Flatten()(attention)
attention = layers.Activation('softmax')(attention)
attention = layers.RepeatVector(64)(attention)
attention = layers.Permute([2, 1])(attention)

multiplied = layers.Multiply()([bidirectional_2, attention])
sent_representation = layers.Dense(512)(multiplied)

tdistributed_1 = layers.TimeDistributed(layers.Dense(40, activation='tanh'))(sent_representation)
tdistributed_2 = layers.TimeDistributed(layers.Dense(10, activation='tanh'))(tdistributed_1)
tdistributed_3 = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(tdistributed_2)


# WE DO NOT NEED IT FOR TRAINING. SO DISCARD.
## Source: https://stackoverflow.com/questions/37743574/hard-limiting-threshold-activation-function-in-tensorflow
def step_activation(x):
    threshold = 0.4
    cond = tf.less(x, tf.fill(value=threshold, dims=tf.shape(x)))
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x)))

    return out

# https://stackoverflow.com/questions/47034692/keras-set-output-of-intermediate-layer-to-0-or-1-based-on-threshold

step_activation = layers.Dense(1, activation=step_activation, name='threshold_activation')(tdistributed_3)



model = Model(input_frame, tdistributed_3)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=0.0001, decay=0.00001)

model.compile(loss='binary_crossentropy', optimizer="rmsprop")

In [ ]:
model.load_weights("bilstm_weights_yaafe_att.h5")

In [ ]:
how_many_step = 40
how_many_repeat = 30

ix_repeat = 0


while (ix_repeat < how_many_repeat):
    ix_repeat += 0
    
    print ("REPEAT:", ix_repeat)
    ix_step = 0
    from_epi = 1
    
    while (ix_step < how_many_step):
        ix_step += 1

        print ("STEP:", ix_step)
        
        #print("relax")
        #time.sleep(2.5) 

        input_array, output_array = create_data_for_supervised ("./amicorpus/*/audio/", 10, 25, from_epi, 
                                                                from_epi+3, True, 12, False, "yaafe" , False, True)
        
        print (np.mean(output_array))
        max_len = 800 # how many frame will be taken
        step = 800 # step size.

        input_array_specified = []
        output_array_specified = []

        for i in range (0, input_array.shape[0]-max_len, step):
            single_input_specified = (input_array[i:i+max_len,:])
            single_output_specified = (output_array[i:i+max_len,:])

            input_array_specified.append(single_input_specified)
            output_array_specified.append(single_output_specified)

        output_array_specified = np.asarray(output_array_specified)
        input_array_specified = np.asarray(input_array_specified)

        try:

            model.fit(input_array_specified, output_array_specified,
                   epochs=2,
                   batch_size=16,
                   shuffle=True)

        except:
            print ("Pass this epoch.")
            pass

        # https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model

        model.save_weights('bilstm_weights_yaafe_att.h5')    

        input_array = []
        output_array = []

        from_epi += 3
    model.save_weights("bilstm_weights_yaafe_att" + str(ix_repeat) + ".h5")    


**1D Convolution combined with BiLSTM**

In [ ]:
import warnings
import keras
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

from keras import layers
from keras import models
from keras import optimizers
from keras.models import Model
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.models import load_model
import time

frame_shape = (800, 59)

## Network Architecture
# https://keras.io/getting-started/sequential-model-guide/

input_frame = keras.Input(frame_shape, name='main_input')

conv1 = layers.Conv1D(800, 9, padding="same")(input_frame)
conv1 = layers.LeakyReLU()(conv1)
conv1_BN = layers.BatchNormalization()(conv1)
conv1_drop = layers.Dropout(0.5)(conv1_BN)

#conv2 = layers.Conv1D(800, 9, padding="same")(conv1_drop)
#conv2 = layers.LeakyReLU()(conv2)
#conv2_BN = layers.BatchNormalization()(conv2)
#conv2_drop = layers.Dropout(0.3)(conv2_BN)


bidirectional_1 = layers.Bidirectional(layers.LSTM(128, return_sequences=True, kernel_initializer='random_normal',
                bias_initializer='zeros'))(conv1_drop)
bidirectional_1 = layers.LeakyReLU()(bidirectional_1)
bidirectional_1_BN = layers.BatchNormalization()(bidirectional_1)
bidirectional_1_drop = layers.Dropout(0.5)(bidirectional_1_BN)

bidirectional_2 = layers.Bidirectional(layers.LSTM(64, return_sequences=True,  kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_1_drop)
bidirectional_2 = layers.LeakyReLU()(bidirectional_2)
bidirectional_2_BN = layers.BatchNormalization()(bidirectional_2)
bidirectional_2_drop = layers.Dropout(0.5)(bidirectional_2_BN)

bidirectional_3 = layers.Bidirectional(layers.LSTM(36, return_sequences=True,  kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_2_drop)
bidirectional_3 = layers.LeakyReLU()(bidirectional_3)
bidirectional_3_BN = layers.BatchNormalization()(bidirectional_3)
bidirectional_3_drop = layers.Dropout(0.5)(bidirectional_3_BN)


tdistributed_1 = layers.TimeDistributed(layers.Dense(40, kernel_initializer='random_normal',
                bias_initializer='zeros'))(bidirectional_3_drop)
tdistributed_1 = layers.LeakyReLU()(tdistributed_1)
tdistributed_1_BN = layers.BatchNormalization()(tdistributed_1)
tdistributed_1_drop = layers.Dropout(0.5)(tdistributed_1_BN)

tdistributed_2 = layers.TimeDistributed(layers.Dense(10, kernel_initializer='random_normal',
                bias_initializer='zeros'))(tdistributed_1_drop)
tdistributed_2 = layers.LeakyReLU()(tdistributed_2)
tdistributed_2_BN = layers.BatchNormalization()(tdistributed_2)
tdistributed_2_drop = layers.Dropout(0.5)(tdistributed_2_BN)


tdistributed_3 = layers.TimeDistributed(layers.Dense(1, activation='sigmoid', kernel_initializer='random_normal',
                bias_initializer='zeros'))(tdistributed_2_drop)


# WE DO NOT NEED IT FOR TRAINING. SO DISCARD.
## Source: https://stackoverflow.com/questions/37743574/hard-limiting-threshold-activation-function-in-tensorflow
def step_activation(x):
    threshold = 0.4
    cond = tf.less(x, tf.fill(value=threshold, dims=tf.shape(x)))
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x)))

    return out

# https://stackoverflow.com/questions/47034692/keras-set-output-of-intermediate-layer-to-0-or-1-based-on-threshold

step_activation = layers.Dense(1, activation=step_activation, name='threshold_activation')(tdistributed_3)



model = Model(input_frame, tdistributed_3)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=0.0001, decay=0.00001)

Nadam = keras.optimizers.Nadam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=0.00001, schedule_decay=0.0004)

model.compile(loss='binary_crossentropy', optimizer="Nadam")

In [ ]:
model.summary()

In [ ]:
model.load_weights("bilstm_cnn_yaafe8.h5")

In [ ]:
how_many_step = 60
how_many_repeat = 15

ix_repeat = 0


while (ix_repeat < how_many_repeat):
    ix_repeat += 1
    
    print ("REPEAT:", ix_repeat)
    ix_step = 0
    from_epi = 2
    
    while (ix_step < how_many_step):
        ix_step += 1

        print ("STEP:", ix_step)
        
        #print("relax")
        #time.sleep(2.5) 
        try:
            input_array, output_array = create_data_for_supervised ("./amicorpus/*/audio/", 10, 25, 
                                                                    from_epi, from_epi+5, True, 12, True, "yaafe" , False, True)

            print (np.mean(output_array))

            max_len = 800 # how many frame will be taken
            step = 800 # step size.

            input_array_specified = []
            output_array_specified = []

            for i in range (0, input_array.shape[0]-max_len, step):
                single_input_specified = (input_array[i:i+max_len,:])
                single_output_specified = (output_array[i:i+max_len,:])

                input_array_specified.append(single_input_specified)
                output_array_specified.append(single_output_specified)

            output_array_specified = np.asarray(output_array_specified)
            input_array_specified = np.asarray(input_array_specified)


            try:
                model.fit(input_array_specified, output_array_specified,
                   epochs=5,
                   batch_size=8,
                   validation_split = 0.2,
                   shuffle=False)

            except ValueError :
                print ("Pass this epoch.")
                pass
            
            model.save_weights('bilstm_cnn_yaafe.h5')    

            input_array = []
            output_array = []
        except IndexError:
            print ("Index Error. Pass this.")
            pass

        from_epi += 5
    model.save_weights("bilstm_cnn_yaafe" + str(ix_repeat) + ".h5")    


--------

**With trained models, we will create prediction.**

In [ ]:
# To get prediction, we need to give k, 800, 59 array to system.
# Our output is like k, 320, 1
# We need to convert it into milisecond version

import more_itertools as mit

def create_prediction(filename, hop, win_len, threshold, lstm_system, featureplan, sr, overlapping = False, feature_extractor="yaafe"):
    
    """"It takes audio file and create prediction via lstm system. If output exceeds
    threshold, we will say there is speaker change.
    
    Arguments:
    filename= Which file will be considered.
    hop: Hop length (we need it for Librosa)
    win_len: Window length (we need it for Librosa)
    threshold: If prediction exceed this value, we will say there is speaker change
    lstm_system: System will create prediction
    featureplan: Which txt will be used for yaafe feature extraction.
    sr: Sample rate of audio input.
    
    Outputs:
    prediction_array: It stores prediction value for each frame
    prediction_array_rav: Ravel version of prediction array. We will use it.
    prediction_array_ms = It stores which milisecond we have speaker change point.
    """
    
    prediction_vector = []
    
    if (feature_extractor=="pyannote"):
        feature_vector = np.load("./pyannote-audio/tutorials/feature-extraction/AMI/" + filename + ".npy")

    if (feature_extractor=="yaafe"):
        
        try:
            feature_vector = np.load("./yaafe_ami_storage/" + filename + ".npy")
        except: 
            feature_vector = create_numpy_for_audio(audio_file="rad_bremen_media/" + filename + ".wav", 
                                                            feature_extractor="yaafe", 
                                                            hop=hop, win_len=win_len, 
                                                            featureplan=featureplan, sr=sr)
    ix_frame = 0
    
    if (overlapping):
        
        while ((ix_frame+799)<feature_vector.shape[0]):        
           
            prediction = lstm_system.predict(np.expand_dims(feature_vector[ix_frame:ix_frame+800], axis=0))
            prediction = prediction.squeeze(axis=2)
            prediction = prediction.squeeze(axis=0)

            prediction_vector.append(prediction)
            
            ix_frame += 200
            
        prediction_vector = np.asarray(prediction_vector)
        prediction_array = np.ravel(prediction_vector)


        prediction_array_sec = []
        prediction_array_msec = []
        prediction_array_average = []
        
        ix_frame_pred = 0

        total_prediction = len(prediction_array)
        
        print (total_prediction)

        prediction_array_average[0:200] = prediction_array[0:200]
        prediction_array_average[200:400] = (prediction_array[200:400]+prediction_array[800:1000]) * 0.5
        prediction_array_average[400:600] = (prediction_array[400:600]+prediction_array[1000:1200]+
                                             prediction_array[1600:1800]) * 0.33
        
        ix_frame = 600
        count = 0
        
        while ((ix_frame+798)<total_prediction):        

            next_frame = ix_frame + (count * 600) 
            try:
                prediction_array_average[ix_frame:ix_frame+200] = (prediction_array[next_frame:next_frame+200]+
                                                                   prediction_array[next_frame+600:next_frame+800]+
                                                                   prediction_array[next_frame+1200:next_frame+1400]+
                                                                   prediction_array[next_frame+1800:next_frame+2000]) * 0.25
            except:
                pass
            ix_frame += 200
            count += 1


        prediction_array = np.asarray(prediction_array_average)

        for pred in prediction_array:

            if (pred > threshold):
                ms_version = float(win_len + (ix_frame_pred * hop)) # milisecond version to represent end point of first embed            
                prediction_array_msec.append(int(ms_version))
                prediction_array_sec.append(ms_version/1000)

            ix_frame_pred += 1


        prediction_array_smooth = []
        for pred in prediction_array_msec:
            if (pred-hop not in prediction_array_msec):
                prediction_array_smooth.append(pred*0.001)


        prediction_array_tenth_ms = np.asarray(prediction_array_msec)/10

        list_cons = [list(group) for group in mit.consecutive_groups(prediction_array_tenth_ms)]

        prediction_array_msec_smooth = []

        for single_list_cons in list_cons:
            prediction_array_msec_smooth.append(np.mean(prediction_array_msec_smooth)*0.01)

        prediction_array_msec_smooth = np.asarray(prediction_array_msec_smooth)

        which_turn = 0

        for single_mean_s in prediction_array_msec_smooth:
            which_turn += 1

            try:
                start_time = float(prediction_array_msec_smooth[which_turn-1])
                end_time = float(prediction_array_msec_smooth[which_turn])

                if ((start_time+0.5) > end_time):
                    prediction_array_msec_smooth[which_turn] = ((start_time+end_time) / 2)
                    prediction_array_msec_smooth = np.delete(mean_s, which_turn-1)
                    which_turn -= 1
            except:
                pass
        
    else:
        while (ix_frame+799<matrix_of_single_audio.shape[0]):        
        
            prediction = lstm_system.predict(np.expand_dims(prediction_vector[ix_frame:ix_frame+800], axis=0))
            prediction = prediction.squeeze(axis=2)
            prediction = prediction.squeeze(axis=0)

            prediction_vector.append(prediction)
            ix_frame += 800
        
        prediction_vector = np.asarray(prediction_vector)
        print (prediction_vector.shape)

        prediction_array = np.ravel(prediction_vector)


        prediction_array_sec = []
        prediction_array_msec = []
        ix_frame_pred = 0

        for pred in prediction_array:
            if (pred > threshold):
                ms_version = float(win_len + (ix_frame_pred * hop)) # milisecond version to represent end point of first embed            
                prediction_array_msec.append(int(ms_version))
                prediction_array_sec.append(ms_version/1000)

            ix_frame_pred += 1

        prediction_array_tenth_ms = np.asarray(prediction_array_msec)/10

        list_cons = [list(group) for group in mit.consecutive_groups(prediction_array_tenth_ms)]

        prediction_array_msec_smooth = []

        for single_list_cons in list_cons:
            prediction_array_msec_smooth.append(np.mean(single_list_cons)*0.01)

        prediction_array_msec_smooth = np.asarray(prediction_array_msec_smooth)

        which_turn = 0

        for single_mean_s in prediction_array_msec_smooth:
            which_turn += 1

            try:
                start_time = float(prediction_array_msec_smooth[which_turn-1])
                end_time = float(prediction_array_msec_smooth[which_turn])

                if ((start_time+0.5) > end_time):
                    mean_s[which_turn] = ((start_time+end_time) / 2)
                    mean_s = np.delete(mean_s, which_turn-1)
                    which_turn -= 1
                    
            except:
                pass

                
    # https://codereview.stackexchange.com/questions/5196/grouping-consecutive-numbers-into-ranges-in-python-3-2

    np.savetxt(fname="./prediction_txt/" + filename + "_prediction.txt", 
               X=prediction_array_msec_smooth, 
               delimiter=' ', fmt='%1.3f')

    return (prediction_array, prediction_array_msec_smooth)


In [ ]:
def txt_file_to_matrix (filename, type_of_text, sr, featureplan, hop=10, win_len=25, feature_extractor="yaafe"):
    
    
    """It takes the reference(ground truth) text file or prediction text file (they are in second version) and 
    return output array which represent the which frames has a speaker change point.
    
    Arguments:
    filename: Which file will be considered.
    type_of_text: Is it prediction or reference file.
    sr: Sample rate.
    featureplan: Which txt will be used for yaafe feature extraction.
    hop: Hop length. (for Librosa.)
    win_len: Window length. (for Librosa)
    feature_extractor: Which feature extractor will be used. (Now, we have 2 option as
        Pyannote or Yaafe.)"""
    
    
    if (feature_extractor=="pyannote"):
        feature_vector = np.load("./pyannote-audio/tutorials/feature-extraction/AMI/" + filename + ".npy")
        
    if (feature_extractor=="yaafe"):
        try:
            feature_vector = np.load("./yaafe_ami_storage/" + filename + ".npy")
        except: 
            feature_vector = create_numpy_for_audio(audio_file="rad_bremen_media/" + filename + ".wav", 
                                                            feature_extractor="yaafe", hop=hop, win_len=win_len, 
                                                            featureplan=featureplan, sr=sr)
    
    
    if (type_of_text == "reference"):
        path_for_txt = "./txt_files/" + filename.split(".")[0] + "_full_time.txt" 
    
    
    if (type_of_text == "prediction"):
        path_for_txt = "./prediction_txt/" + filename + "_prediction.txt"
        
    change_seconds = []


    with open(path_for_txt) as f:
        content = f.readlines()
        
    content = [x.strip() for x in content] 

    for single_change in change_seconds:

        change_seconds.append(single_change)

    output_array = np.zeros(feature_vector.shape[0])

    for single_change in change_seconds:
        single_change_ms = float(single_change)*1000
        which_start_hop = (single_change_ms-win_len)/hop # now we know, milisecond version of change
                                    # which is located after which_hop paramater
                                    # add 2 and round to up
        which_end_hop = end_time_ms/hop # round to up

        start_location = math.ceil(which_start_hop + 1)
        end_location = math.ceil(which_end_hop)


        output_array[start_location:end_location+1] = 1.0

    return (output_array)

-----

**We need to evaluate our results**

In [ ]:
import matplotlib.pyplot as pp
pp.rcParams['figure.figsize'] = (50.8, 10.0)
%matplotlib inline

In [ ]:
# Evaluation Metric
from pyannote.metrics.diarization import DiarizationPurityCoverageFMeasure
from pyannote.metrics.segmentation import SegmentationPurity
from pyannote.metrics.segmentation import SegmentationCoverage
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.database import get_protocol
from pyannote.core import Segment, Timeline, Annotation


metric = DiarizationPurityCoverageFMeasure()
seg_purity = SegmentationPurity()
seg_coverage = SegmentationCoverage()

def complete_evaluation_single_file(filename, hop, win_len, thres, model, sr, featureplan, overlapping = False, show_graph_frame=True, 
                                    feature_extractor="yaafe"):
    
    """With this function, we can get different metrics to evaluate the system's performance.
    
    Arguments:
    filename: Which file will be considered.
    hop: Hop length.
    win_len: Window length.
    thres:
    model: Which Deep Learning model will be used to predict.
    sr: Sample rate of input.
    featureplan:
    overlapping: Will we use overlapping-averaging to predict output.
    show_graph_frame: If it is true, we can see the prediction
        and ground truth at the figure.
    feature_extractor: Which feature extractor will be used. (currently, 
        Pyannote and Yaafe are available.)"""
    
    prediction_array, prediction_array_msec = create_prediction(filename, hop=hop, win_len=win_len, 
                                                                threshold = thres, lstm_system=model,
                                                                feature_extractor=feature_extractor, sr=sr,
                                                                featureplan=featureplan,
                                                                overlapping=overlapping)
    
    ground_truth = txt_file_to_matrix(filename, "reference", sr, featureplan, 
                                      hop=hop, win_len=win_len, feature_extractor=feature_extractor)
    
    prediction_output_array = txt_file_to_matrix(filename, "prediction", sr, featureplan, 
                                                 hop=hop, win_len=win_len, feature_extractor=feature_extractor)
    
    path_for_reference = "./txt_files/" + filename.split(".")[0] + "_full_time.txt"
   
        
    path_for_prediction = "./prediction_txt/" + filename + "_prediction.txt"
   
        
    with open(path_for_reference) as f:
        change_seconds_ref = f.readlines()
        
    change_seconds_ref = [x.strip() for x in change_seconds_ref] 
    
    change_miliseconds_ref = [int(1000*float(i)) for i in change_seconds_ref]
    
    x_axis = np.arange(1, len(ground_truth)+1)
    
    if (show_graph_frame):
    
        pp.rcParams['figure.figsize'] = (50.8, 10.0)
        pp.plot(prediction_array[0:20000])
        pp.plot(x_axis[0:20000], ground_truth[0:20000], 'x', color='black');
        pp.plot(x_axis[0:20000], 0.95*prediction_output_array[0:20000], '.', color="pink");

        pp.axhline(y=thres, color='r', linestyle='-')
        pp.show()

    hypothesis = Annotation()

    which_turn = 0
    
    with open(path_for_prediction) as f:
        change_seconds_pred = f.readlines()
        
    change_seconds_pred = [x.strip() for x in change_seconds_pred] 
        
    for single_change_pred in change_seconds_pred:
        try:
            which_turn += 1
            start_time = change_seconds_pred[which_turn-1]
            end_time = change_seconds_pred[which_turn]
            hypothesis[Segment(float(start_time), float(end_time))]="a"
        except:
            pass
   
    change_seconds_pred = [x.strip() for x in change_seconds_pred] 
    change_miliseconds_pred = [int(1000*float(i)) for i in change_seconds_pred]
    
    
    total_change_ref = len(change_miliseconds_ref)
    
    total_change_pred = len(change_miliseconds_pred)
    
    correct_guess = 0

    for single_change_ref in change_miliseconds_ref:
        single_guess_mem = False 
        for single_change_pred in change_miliseconds_pred:
            if (single_change_pred in range(single_change_ref-500, single_change_ref+500)):
                correct_guess += 1
                break
            
    print ("Correst Guess: ", correct_guess)    
    print ("Total End Time: ", total_end_time_num)    
    print ("Total Guess: ", total_guess_num)   

    # Calculate precision, recall and f1 score.
    precision = correct_guess / float(total_guess_num)
    recall = correct_guess / float(total_end_time_num)
    
    try:
        f1 =  2 * (precision*recall) / (precision+recall)
    except ZeroDivisionError:
        f1 = 0
    
    print ("precison: ", precision)
    print ("recall: ", recall)
    print ("f1: ", f1)
    
    
    purity = 0
    coverage = 0
    
    try:
        # If file in the Ami Corpus, we can get the result from
        # pyannote.metrics.
        
        protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset')

        for i in protocol.train():
            if (i["uri"] == filename):
                 reference = i['annotation']

        for i in protocol.test():
            if (i["uri"] == filename):             
                reference = i['annotation']

        for i in protocol.development():
            if (i["uri"] == filename):             
                reference = i['annotation']

        metric = SegmentationPurity()
        purity = metric(reference, hypothesis)
        print ("Segmentation Purity: ", purity)

        metric = SegmentationCoverage()
        coverage = metric(reference, hypothesis)
        print ("Segmentation Coverage: ", coverage)


    except:
        pass
    
    return (precision, recall, f1, purity, coverage)

In [ ]:
complete_evaluation_single_file(filename="TV_bubi6_2017-04-11_gek", hop=10, win_len=25, thres = 0.25, 
                               model=model, feature_extractor="yaafe",
                               sr=16000, featureplan = "featureplan_new.txt",
                               overlapping=False)

In [ ]:
complete_evaluation_single_file(filename="TV_3nach9", hop=10, win_len=25, thres = 0.3, 
                               model=model, feature_extractor="yaafe",
                               sr=16000, featureplan = "featureplan_new.txt",
                               overlapping=True)

In [ ]:
complete_evaluation_single_file(filename="TV_Wetter_2017-04-10", hop=10, win_len=25, thres = 0.15, 
                               model=model, feature_extractor="yaafe",
                               sr=16000, featureplan = "featureplan_new.txt",
                               overlapping=True)

In [ ]:
complete_evaluation_single_file(filename="TS3003a.Mix-Headset", hop=10, win_len=25, thres = 0.18, 
                               model=model, feature_extractor="yaafe",
                               sr=16000, featureplan = "featureplan_new.txt",
                               overlapping=True)

In [ ]:
complete_evaluation_single_file(filename="Wie_ist_nachhaltiger_Tourismus_moeglich_2017-09-16", hop=10, win_len=25, 
                               thres = 0.1, model=model, feature_extractor="yaafe",
                               sr=16000, featureplan = "featureplan_new.txt",
                               overlapping=False)

In [ ]:
## TO GET ALL RESULT FROM ALL CORPUS

precision_array = []
recall_array = []
f1_array = []
purity_array = []
coverage_array = []

corpus = os.listdir("/home/herdogan/Desktop/SpChangeDetect/df_media/")

for file in corpus:
    try:
        filename = file.split(".")[0]
        print (filename)
        
        precision, recall, f1, _, _ = complete_evaluation_single_file(filename, hop=10, win_len=25, 
                                                                      feature_extractor="yaafe", model=model, show_graph_frame=False, 
                                                                      thres=0.12, sr=16000, featureplan="featureplan_new.txt", 
                                                                      overlapping=False)
        precision_array.append(precision)
        recall_array.append(recall)
        f1_array.append(f1)
        
        
        
    except FileNotFoundError:
        print (filename + " is not in the directory")
        pass

In [ ]:
## TO GET ALL RESULT FROM ALL CORPUS

precision_array = []
recall_array = []
f1_array = []
purity_array = []
coverage_array = []

corpus = os.listdir("/home/herdogan/Desktop/SpChangeDetect/txt_ami_full/")

for file in corpus:
    try:
        filename = (file.split(".")[0]).split("_")[0]
        filename = filename + ".Mix-Headset"
        print (filename)
        
        precision, recall, f1, purity, coverage = complete_evaluation_single_file(filename, hop=10, win_len=25, 
                                                                                  feature_extractor="yaafe", model=model, 
                                                                                  show_graph_frame=False, 
                                                                                  thres=0.168, sr=16000, 
                                                                                  featureplan="featureplan_new.txt", overlapping=True)
        precision_array.append(precision)
        recall_array.append(recall)
        f1_array.append(f1)
        purity_array.append(purity)
        coverage_array.append(coverage)
        
    except FileNotFoundError:
        print (filename + " is not in the directory")
        pass

In [ ]:
corpus = os.listdir("/home/herdogan/Desktop/SpChangeDetect/amicorpus/*/")


In [ ]:
print(np.mean(precision_array))
print(np.mean(recall_array))
print(np.mean(f1_array))
print(np.mean(purity_array))
print(np.mean(coverage_array))

In [ ]:
cd rad_bremen_media/

In [ ]:
corpus = os.listdir("/home/herdogan/Desktop/SpChangeDetect/rad_bremen_media/")

for file in corpus:
    print ((file.split(".")[0]))
    new_file = ((file.split(".")[0])) + "RR.wav" 
    !sox $file -r 16000 $new_file


### Create Subsequences with Label

At that point, we should create training and test data with their label. Also, we can use directly [pyannote.metrics](https://github.com/pyannote/pyannote-metrics)

### Deep Learning Architecture

We can directly upload the model's architecture from the .yml file which is provided by writer.

However, I want to directly write all steps.

In [ ]:
# Author's .yml files

!wget https://raw.githubusercontent.com/yinruiqing/change_detection/master/model/architecture.yml

In [ ]:
# Load to model

from keras.models import model_from_yaml
yaml_file = open('architecture.yml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)

In [ ]:
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer="rmsprop")

In [ ]:
import keras
print (keras.__version__)

In [ ]:
model.summary()

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
import keras
from keras.models import Model
import tensorflow as tf
from keras.layers.advanced_activations import *
from keras.utils.generic_utils import get_custom_objects


frame_shape = (320, 35)

## Network Architecture

input_frame = keras.Input(frame_shape, name='main_input')

bidirectional_1 = layers.Bidirectional(layers.LSTM(32, return_sequences=True))(input_frame)
bidirectional_2 = layers.Bidirectional(layers.LSTM(20, activation='tanh', return_sequences=True))(bidirectional_1)

tdistributed_1 = layers.TimeDistributed(layers.Dense(40, activation='tanh'))(bidirectional_2)
tdistributed_2 = layers.TimeDistributed(layers.Dense(10, activation='tanh'))(tdistributed_1)
tdistributed_3 = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(tdistributed_2)


# WE DO NOT NEED IT FOR TRAINING. SO DISCARD.
## Source: https://stackoverflow.com/questions/37743574/hard-limiting-threshold-activation-function-in-tensorflow
def step_activation(x):
    threshold = 0.4
    cond = tf.less(x, tf.fill(value=threshold, dims=tf.shape(x)))
    out = tf.where(cond, tf.zeros(tf.shape(x)), tf.ones(tf.shape(x)))

    return out

# https://stackoverflow.com/questions/47034692/keras-set-output-of-intermediate-layer-to-0-or-1-based-on-threshold

step_activation = layers.Dense(1, activation=step_activation, name='threshold_activation')(tdistributed_3)



model = Model(input_frame, tdistributed_3)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss='binary_crossentropy', optimizer="rmsprop")

In [ ]:
model.summary()

In [ ]:
# To save our model

model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

In [ ]:
# To look our model

!cat model.yaml